# Silicone
### Erica Simon, 02/13/24
## Purpose: use the Silicone infilling tool to fill missing emissions data for certain species
Credit: 
- Lamboll, R. D., Nicholls, Z. R. J., Kikstra, J. S., Meinshausen, M., & Rogelj, J. (2020). Silicone v1.0.0: an open-source Python package for inferring missing emissions data for climate change research. *Geoscientific Model Development, 13(11),* 5259–5275. https://doi.org/10.5194/gmd-13-5259-2020
- Meinshausen, M., Lewis, J., McGlade, C. et al. Realization of Paris Agreement pledges may limit warming just below 2 °C. *Nature* 604, 304–309 (2022). https://doi.org/10.1038/s41586-022-04553-z
    - code at https://github.com/climate-resource/ndc-realisations-2021





In [1]:
import pandas as pd
import numpy as np
import pyam

import silicone.multiple_infillers as mi
import silicone.database_crunchers as cr

<IPython.core.display.Javascript object>

In [2]:
import silicone.database_crunchers
from silicone.time_projectors import ExtendLatestTimeQuantile
import scmdata
import scmdata.database
import matplotlib.pyplot as plt

from tqdm.autonotebook import tqdm

/opt/miniconda/envs/risk/lib/python3.10/site-packages/scmdata/database/_database.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tqdman


In [3]:
future_df = pd.read_csv('~/outputs/GCAM_infilling.csv')
hist_df = pd.read_csv('~/outputs/hist_emis_ALL.csv')

df_to_infill = pyam.IamDataFrame(future_df)
df = pyam.IamDataFrame(hist_df)

/opt/miniconda/envs/risk/lib/python3.10/site-packages/pyam/utils.py:316: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.set_index(index + REQUIRED_COLS + extra_cols)
/opt/miniconda/envs/risk/lib/python3.10/site-packages/pyam/utils.py:316: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.set_index(index + REQUIRED_COLS + extra_cols)


In [4]:
missing_vars = np.setdiff1d(hist_df['Variable'].unique(), future_df['Variable'].unique())

In [5]:
lead = ['Emissions|CO2 FFI']
variables_of_interest = ['Emissions|C3F8']
years_list = list(range(2022, 2101))

In [6]:
unavailable_variables = [
        variab for variab in variables_of_interest if variab not in df.variable
    ]


In [7]:
unavailable_variables

[]

In [8]:
df

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : Historical (1)
 * scenario : GCP+CEDS+PRIMAP+GFED (1)
Timeseries data coordinates:
   region   : World (1)
   variable : Emissions|BC, Emissions|C2F6, Emissions|C3F8, ... Emissions|c-C4F8 (51)
   unit     : Gt CO2/yr, Mt BC/yr, Mt CH4/yr, Mt CO/yr, Mt N2O/yr, ... kt cC4F8/yr (50)
   year     : 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, ... 2022 (273)

In [9]:
df_to_infill

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : GCAM 6.0 NGFS (1)
 * scenario : Below 2 C, Current Policies, Delayed transition, ... Net Zero 2050 (7)
Timeseries data coordinates:
   region   : World (1)
   variable : Emissions|BC, Emissions|C2F6, Emissions|CF4, ... Emissions|VOC (21)
   unit     : Gt CO2/yr, Mt BC/yr, Mt CH4/yr, Mt CO/yr, Mt N2O/yr, ... kt SF6/yr (20)
   year     : 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, ... 2100 (81)

In [10]:
df_infilled = mi.infill_all_required_variables(
    df_to_infill,
    df,
    variable_leaders=lead,
    required_variables_list=variables_of_interest, # If None, would infill a default list
    cruncher=cr.QuantileRollingWindows,
    output_timesteps=years_list,
    infilled_data_prefix=None,
    to_fill_old_prefix=None,
    check_data_returned=False,
)

Filling required variables:   0%|          | 0/1 [00:00<?, ?it/s]/opt/miniconda/envs/risk/lib/python3.10/site-packages/silicone/database_crunchers/quantile_rolling_windows.py:182: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  wide_db = wide_db.applymap(lambda x: np.nan if isinstance(x, str) else x)
/opt/miniconda/envs/risk/lib/python3.10/site-packages/pyam/core.py:895: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  self.meta[name] = meta[name].combine_first(self.meta[name])
/opt/miniconda/envs/risk/lib/python3.10/site-packages/silicone/multiple_infillers/infill_all_required_emissions_for_openscm.py:299: UserWarning: Error encountered when infilling Emissions|C3F8
  warnings.warn(f"Error encountered when infilling {req_var}")
Filling 

ValueError: Not all required timepoints are present in the database we crunched, we crunched 
	`[2022]`
but you passed in 
	[2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100]

In [ ]:
df_infilled.filter(variable=variables_of_interest[0]).timeseries().head()

In [ ]:
df_to_infill.filter(variable=variables_of_interest[0]).timeseries().head()

In [ ]:
extended_scenario_all = scmdata.run_append(
    [
        extend_timeseries(df_to_infill, selected_scenarios)
        # for p in tqdm(pathways)
    ]
)
# extended_scenario_2050 = extended_scenario_all.filter(pathway_id=SELECTED_PATHWAY)

In [ ]:
from ndcs.infilling import extend_timeseries